In [2]:
from sklearn.model_selection import KFold
from lightgbm import log_evaluation, early_stopping
import lightgbm as lgb
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")



def del_PART_LIST(PART_LIST):
    """
    零部件信息表
    :param PART_LIST:
    :return:
    """
    part_list = pd.read_csv(PART_LIST, encoding="GBK")
    part_list['PIDT'] = pd.to_datetime(part_list['PIDT']).dt.strftime('%Y-%m-%d')
    ONEHOT_COLUMNS = ['CAR_CLASS', 'REPAIR_TYPE', 'TYPE_CODE', 'CONSTRUCT_NAME']

    DROP = ['KPDS', 'PIDT']
    for column in ONEHOT_COLUMNS:
        part_list = hot_coding_dispose(part_list, column)

    part_list = part_list.drop(columns=DROP)
    # 去重
    part_list = part_list.drop_duplicates(['PART_NO'],keep='last')

    return part_list


def del_PART_ORDER(PART_ORDER):
    """
    订单表
    :return:
    """

    part_order = pd.read_csv(PART_ORDER).fillna(0)


    part_order['MONTH'] = pd.to_datetime(part_order['MONTH']).dt.strftime('%Y-%m-%d')
    part_order['MONTH_INT'] = part_order.apply(month_int_time, axis=1)
    # 去重
    part_order = part_order.drop_duplicates(subset=['PART_NO', 'MONTH_INT'], keep='last')
    # 排序
    part_order = part_order.sort_values(by='MONTH_INT')

    # print(part_order)

    return part_order


def del_TESTDATA_ID(TESTDATA_ID):
    """
    预测表
    :param TESTDATA_ID:
    :return:
    """
    part_predict_list = []
    with open(TESTDATA_ID, 'r') as f:
        for part in f.readlines():
            part_predict_list.append(part[:-1])

    return part_predict_list




def month_int_time(a):
    """
    从2015-1-1开始当前为第sum个月
    :param a:
    :return:
    """
    month = int(a['MONTH'].split('-')[1]) - 1
    year = (int(a['MONTH'].split('-')[0]) - 2015 ) * 12
    sum = year + month
    return sum

# def hot_encode():


def hot_coding_dispose(data, column):
    """
    使用热编码完成字符串转化
    :param data: 数据集
    :param column: 需要编码和删除的列名
    :return: 合并之后的集合
    """
    # 使用热门编码转化字符串
    code = pd.DataFrame({column: data[column]})
    code_DataFrame = pd.get_dummies(code)
    # merge，改用concat
    data = pd.concat([data, code_DataFrame], axis=1)
    # 删除列
    data = data.drop(columns=column)

    return data

